In [6]:
from EDA.DataReader import DataReader
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
import torch 
from torcheval.metrics.functional import multiclass_f1_score
import warnings

In [7]:
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

In [8]:
def custom_tokenizer(text):
    return text.split()

In [9]:
# pre processing data
dataset = DataReader("UIT_VFSC") # UIT
# dataset = DataReader("") # dataset foody_raw
df_train = dataset.df_train
df_test = dataset.df_test
df_total =dataset.df_total
n_labels = int(df_total["label"].max().item() + 1)

In [10]:
cv = CountVectorizer(tokenizer=custom_tokenizer, stop_words=[])
X = cv.fit_transform(df_train["corpus"])
x_test = cv.transform(df_test["corpus"])

In [11]:
# Combine with sklearn random forest
mod = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=None, criterion='gini', max_features='sqrt')
mod.fit(X, df_train["label"])
y_pred = mod.predict(x_test)
accur = accuracy_score(df_test["label"], y_pred)
print(f"Accuracy: {accur:.4f}")

y_pred = torch.tensor(y_pred).type(torch.long)
label = torch.tensor(df_test["label"].tolist()).type(torch.long)
mf1 = multiclass_f1_score(y_pred, label, num_classes=n_labels, average='macro')
wf1 = multiclass_f1_score(y_pred, label, num_classes=n_labels, average='weighted')
print(f"F1-Score: {max(mf1, wf1):.4f}")


Accuracy: 0.8781
F1-Score: 0.8691
